In [1]:
import os, sys
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from Features_Fourier import dft_features

### Loading Data

In [2]:
# Load data configuration
data_dir = r'C:\Users\alail\OneDrive\Documentos\dado_iniciais'
pen = [10, 50, 75]

df_cases = pd.read_csv(
    os.path.join(data_dir, 'cenarios_falta_preprocessamento.csv'),
    header=None,
    index_col=False
)

In [3]:
# Load data
if os.path.exists(os.path.join(data_dir, 'data.npy')):
    data = np.load(os.path.join(data_dir, 'data.npy'))
else:
    one_data_size = (6, 3072)
    data = np.zeros((len(pen) * len(df_cases), *one_data_size))
    for i, p in enumerate(pen):
        for j, case in df_cases.iterrows():
            data[i * len(df_cases) + j] = np.loadtxt(
                os.path.join(data_dir, f'pen{p}', f'sinal_{j+1}_.csv'),
                delimiter=','
            ).T
    np.save(os.path.join(data_dir, 'data.npy'), data) # save in numpy for faster loading

In [8]:
# Create a Dataframe to store the features

columns_name = [
    'File ID',
    'Fault type',
    'Fault resistance', 
    'Incidence angle',
    'Fault location',
    'Fault distance',
    'Pen',
    'Fourier Feature',
    'Wavelet Feature',
    'Stockwell Feature',
    'Hilbert-Huang Feature'
]

columns_type = {
    'File ID': int,
    'Fault type': int,
    'Fault resistance': float,
    'Incidence angle': float,
    'Fault location': int,
    'Fault distance': float,
    'Pen': int,
    'Fourier Feature': object,
    'Wavelet Feature': object,
    'Stockwell Feature': object,
    'Hilbert-Huang Feature': object
}

df_features = pd.DataFrame(
    columns= columns_name,
    index=range(len(pen) * len(df_cases))
)

for i, p in enumerate(pen):
    for j, case in df_cases.iterrows():
        df_features.loc[i * len(df_cases) + j] = case.tolist() + [p, [], [], [], []]
df_features = df_features.astype(columns_type)
del df_cases

### Fourier Features

In [ ]:
fs = 512*60
for i in tqdm(range(data.shape[0])):
    df_features.loc[i, 'Fourier Feature'] = dft_features(data[i], fs)
    break

  0%|          | 0/4050 [00:00<?, ?it/s]

### Wavelet Features

In [ ]:
x = df_features.loc[0, 'Fourier Feature']



### Save Dataframe

In [ ]:
df_features.to_pickle(os.path.join(data_dir, 'features.pkl'))